## Ising Model Simulation and Bayesian Inference

## Libraries

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from itertools import product
from collections import Counter
import pickle
import os
os.chdir("../")

## Sampling from Ising Model

### Functions

In [13]:
# Define the Ising model energy function
def spin_energy(spins, i,j, N, J, h):
    E_interact = 0
    E_field = 0

    target_spin = spins[i,j]

    neigh_coords = [(i-1, j), (i, j-1), (i+1, j), (i, j+1)]
    neigh_coords = [coord for coord in neigh_coords 
                    if (coord[0] in range(N) and coord[1] in range(N))]
    
    neigh_spins = [spins[coord[0], coord[1]] for coord in neigh_coords]

    for neigh_spin in neigh_spins:
        E_interact -= J * target_spin * neigh_spin   # Nearest neighbor interactions
    
    E_field -= h * spins[i, j]  # External field

    return E_interact, E_field


def total_energy(spins, N, J, h):

    for i in range(N):
        for j in range(N):
            E_interact, E_field = spin_energy(spins, i, j, N, J, h)

    return E_interact + E_field

def generate_state(N, state_index):
    """Generate a specific spin configuration for a lattice of size N x N"""
    state_binary = np.binary_repr(state_index, width=N**2)
    state_spins = [-1 if spin == '0' else 1 for spin in state_binary]
    state = np.array(state_spins).reshape(N, N)
    return state

def generate_all_states(N):
    """Generate all possible spin configurations for a lattice of size N x N"""
    num_states = 2 ** (N ** 2)  # Total number of states
    os.makedirs(f"data/processed/ising_states/{N}/", exist_ok=True)
    

    # Generate and save states one by one
    for state_index in range(num_states):
        state = generate_state(N, state_index)

        pickle_filename = f"state_{state_index}.pkl"
        pickle_filepath = os.path.join(f"data/processed/ising_states/{N}/", pickle_filename)

        with open(pickle_filepath, "wb") as pickle_file:
            pickle.dump(state, pickle_file)

    return None

def calculate_probabilities_and_energies(N, J, h, T):
    """Calculate the probability and energy of each spin configuration"""
    probabilities = []
    energies = []
    Z = 0  # Partition function

    num_states = 2 ** (N ** 2)

    for state_index in range(num_states):

        state = generate_state(N, state_index)

        energy = total_energy(state, N, J, h)
        prob = np.exp(-energy / T)
        Z += prob
        probabilities.append(prob)
        energies.append(energy)

    probabilities = [prob / Z for prob in probabilities]  # Normalize probabilities
    return probabilities, energies

def order_states_by_probability(probabilities, energies):
    """Order the states based on their probabilities in descending order"""

    states = list(range(len(probabilities)))  # State indices
    state_prob_energy_pairs = list(zip(states, probabilities, energies))
    state_prob_energy_pairs.sort(key=lambda x: x[1], reverse=True)
    ordered_states = [pair[0] for pair in state_prob_energy_pairs]
    ordered_probabilities = [pair[1] for pair in state_prob_energy_pairs]
    ordered_energies = [pair[2] for pair in state_prob_energy_pairs]
    return ordered_states, ordered_probabilities, ordered_energies

def metropolis_hastings(ordered_states, ordered_probabilities, num_steps):
    """Perform the Metropolis-Hastings simulation using pre-computed states, probabilities, and energies"""
    num_states = len(ordered_states)
    current_state_index = 0  # Start with the first state
    states = [current_state_index]  # Initialize with the current state index
    transitions = np.zeros((num_states, num_states))  # Initialize the transition matrix

    for step in range(num_steps):
        proposed_state_index = np.random.randint(num_states)  # Propose a new state index
        acceptance_probability = min(1, ordered_probabilities[proposed_state_index] / ordered_probabilities[current_state_index])

        if np.random.rand() < acceptance_probability:
            # Accept the proposed state
            current_state_index = proposed_state_index

        states.append(current_state_index)
        transitions[current_state_index, proposed_state_index] += 1  # Update the transition matrix

    return states, transitions

In [ ]:
# Define the Ising model parameters
N = 200  # Lattice size
J = 1.0  # Interaction coefficient
h = 1.0  # External field
T = 2.0  # Temperature
num_steps = 100000  # Number of steps in the Metropolis-Hastings simulation

In [16]:
generate_all_states(N)

OSError: [Errno 28] No space left on device: 'data/processed/ising_states/20/state_1762474.pkl'